In [2]:
import os
import pkg_resources
import whitebox
from osgeo import gdal
from osgeo import gdal_array
import numpy as np

wbt = whitebox.WhiteboxTools()
print(wbt.version())
print(wbt.help())

work_dir = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\LOCAL30\COSTS"
wbt.set_working_dir(work_dir)
wbt.verbose = False

Decompressing WhiteboxTools_win_amd64.zip ...
WhiteboxTools package directory: C:\Users\vince\anaconda3\envs\conn\lib\site-packages\whitebox
WhiteboxTools v2.1.0 by Dr. John B. Lindsay (c) 2017-2021

WhiteboxTools is an advanced geospatial data analysis platform developed at
the University of Guelph's Geomorphometry and Hydrogeomatics Research 
Group (GHRG). See www.whiteboxgeo.com for more details.

WhiteboxTools Help

The following commands are recognized:
--cd, --wd       Changes the working directory; used in conjunction with --run flag.
-h, --help       Prints help information.
-l, --license    Prints the whitebox-tools license. Tool names may also be used, --license="Slope"
--listtools      Lists all available tools. Keywords may also be used, --listtools slope.
-r, --run        Runs a tool; used in conjunction with --wd flag; -r="LidarInfo".
--toolbox        Prints the toolbox associated with a tool; --toolbox=Slope.
--toolhelp       Prints the help associated with a tool; --too

In [3]:
# INPUTS

cost_30 = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\INPUTS\cost_30.tif"
clumps_30 = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\INPUTS\clumps_30.tif"
cents_30 = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\INPUTS\centroids_30.tif"

core_30 = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\INPUTS\core_30.tif"
clu30 = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\INPUTS\core_30_clump.tif"

In [ ]:
# 50 Acres

work_dir = None
work_dir = r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\LOCAL30\50_3"
wbt.set_working_dir(work_dir)

wbt.raster_area(clu30,"areas.tif",False,"grid cells",True)
wbt.greater_than("areas.tif",225,"acres50.tif") # 225 cells is 50 Acres
wbt.multiply(clu30,"acres50.tif","clumps50.tif")

clumps50 = gdal.Open(r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\LOCAL30\50_3\clumps50.tif")
v =  clumps50.GetRasterBand(1)
w = np.array(v.ReadAsArray())
x = np.unique(w)
y = np.delete(x,[0,1])
z = y.astype(int)
print(z)

wbt.cost_distance("clumps50.tif",cost_30,"aaa.tif","bbb.tif")
wbt.cost_allocation("clumps50.tif","bbb.tif","allocation.tif")

wbt.multiply(clu30,0,"zero.tif")
wbt.add("zero.tif",1000000000000,"corridor.tif")

for i in z:
    qu = None
    band1 = None
    array1 = None
    values1 = None
    wbt.equal_to("clumps50.tif",i,"clump"+str(i)+".tif")
    wbt.cost_distance("clump"+str(i)+".tif",cost_30,"a"+str(i)+".tif","None.tif")
    wbt.equal_to("allocation.tif",i,"q.tif")
    wbt.convert_nodata_to_zero("q.tif","qui.tif")
    wbt.buffer_raster("qui.tif","qbuff.tif",1,gridcells=True)
    wbt.multiply("qbuff.tif","allocation.tif","mult.tif")
    qu = gdal.Open(r"C:\Rhumb Line Maps Dropbox\Contracts\MidcoastConservancy\18_Connectivity\CONNECTIVITY\LOCAL30\50_3\mult.tif")
    band1 =  qu.GetRasterBand(1)
    array1 = np.array(band1.ReadAsArray())
    values1 = np.unique(array1)
    rightvalues = None
    rightvalues = np.delete(values1,[0,1])
    numb = None
    numb, = np.where(rightvalues == i)
    vals2 = None
    vals2 = np.delete(rightvalues,numb)
    vals3 = None
    vals3 = vals2.astype(int)
    os.remove(os.path.join(work_dir,"q.tif"))
    os.remove(os.path.join(work_dir,"qui.tif"))
    os.remove(os.path.join(work_dir,"qbuff.tif"))
    qu = None
    os.remove(os.path.join(work_dir,"mult.tif"))
    for j in vals3:
        wbt.equal_to("clumps50.tif",j,"clump"+str(j)+".tif")
        wbt.cost_distance("clump"+str(j)+".tif",cost_30,"a"+str(j)+".tif","None.tif")
        wbt.add("a"+str(i)+".tif","a"+str(j)+".tif","a"+str(i)+"_"+str(j)+".tif")
        summ = None
        summ = gdal_array.LoadFile(os.path.join(work_dir,"a"+str(i)+"_"+str(j)+".tif"))
        summ = np.ma.masked_equal(summ, -3.4028234663852886e+38)
        summ = np.ma.filled(summ, np.nan)
        minn = None
        minn = np.nanpercentile(summ,0)
        summ = None
        wbt.subtract("a"+str(i)+"_"+str(j)+".tif",minn,"b"+str(i)+"_"+str(j)+".tif")
        wbt.min("corridor.tif","b"+str(i)+"_"+str(j)+".tif","newcorridor.tif")
        if os.path.exists(os.path.join(work_dir,"newcorridor.tif")):
            os.remove(os.path.join(work_dir,"corridor.tif"))
            if os.path.exists(os.path.join(work_dir,"corridor.tif"))==False:
                os.rename(os.path.join(work_dir,"newcorridor.tif"),os.path.join(work_dir,"corridor.tif"))     
                os.remove(os.path.join(work_dir,"a"+str(i)+"_"+str(j)+".tif"))
                os.remove(os.path.join(work_dir,"b"+str(i)+"_"+str(j)+".tif"))